<a href="https://colab.research.google.com/github/Anikrage/Visual_Product_Matcher/blob/main/TL_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pymongo python-dotenv tensorflow Pillow requests tqdm -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 29.9 MB/s eta 0:00:00


In [5]:
import os
import requests
import numpy as np
from PIL import Image
from io import BytesIO

In [6]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array

In [7]:
from pymongo import MongoClient
from pymongo.server_api import ServerApi

In [8]:
from google.colab import userdata

Databse Init

In [9]:
username = userdata.get('MONGO_USERNAME')
password = userdata.get('MONGO_PASSWORD')
cluster = userdata.get('MONGO_CLUSTER')
database = userdata.get('MONGO_DATABAS')

In [10]:
db_url = f"mongodb+srv://{username}:{password}@{cluster}/?retryWrites=true&w=majority"

In [11]:
client = MongoClient(db_url, server_api=ServerApi('1'))
try:
    client.admin.command('ping')
    print("Connected")
except Exception as e:
    print(e)

Connected


In [12]:
db=client[database]
products_collection=db['products']

Model Init

In [13]:
model=ResNet50(
    weights='imagenet',include_top=False,
    pooling='avg',input_shape=(224,224,3)
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [14]:
print(f"Output shape: {model.output_shape}")

Output shape: (None, 2048)


In [65]:
def get_image_features(img_url):
  try:
    response = requests.get(img_url, timeout=10)
    img=Image.open(BytesIO(response.content))

    img = img.convert('RGB')
    img = img.resize((224,224))
    img_array=img_to_array(img)
    img_array=np.expand_dims(img_array,axis=0)
    img_array=preprocess_input(img_array)
    features=model.predict(img_array,verbose=0)
    features=features.flatten()
    normalized=np.linalg.norm(features)
    if normalized > 0:
      features=features/normalized

    return features
  except Exception as e:
    print(f"Errror: {e}")

Testing on 1 product

In [66]:
test=products_collection.find_one({'image_url':{'$exists':True}})
print(test['name'])

Basics Men Blue Striped T-shirt


In [67]:
test_features=get_image_features(test['image_url'])
print(test_features)

[0.00191086 0.0086785  0.00712817 ... 0.00056408 0.00176883 0.02023942]


Extracting Features of all products in DB

In [68]:
products=products_collection.find({'image_url':{'$exists':True}})
products=list(products)

In [24]:
from tqdm import tqdm

In [69]:
for i,p in tqdm(enumerate(products),total=len(products)):
  if 'feature_embed' in p:
    continue
  image_url=p['image_url']
  features=get_image_features(image_url)
  if features is not None:
    feature_list=features.tolist()
    products_collection.update_one(
        {'_id':p['_id']},
        {'$set':{'embedding':feature_list}}
    )

100%|██████████| 225/225 [01:39<00:00,  2.26it/s]


testing on irl data

In [97]:
test_url=test['image_url']
test_features=get_image_features(test_url)

In [98]:
test_embed=test_features.reshape(1,-1)

In [99]:
db_embeds=products_collection.find({'embedding':{'$exists':True}})
db_embeds=list(db_embeds)

In [100]:
from sklearn.metrics.pairwise import cosine_similarity

In [101]:
matches=[]
for p in db_embeds:
  p_embed=np.array(p['embedding']).reshape(1,-1)
  similar=cosine_similarity(test_embed,p_embed)

  matches.append({'name':p['name'],
                  'image_url':p['image_url'],
                  'similarity':similar})

In [102]:
matches.sort(key=lambda x:x['similarity'],reverse=True)

In [103]:
print(matches[0])

{'name': 'Basics Men Blue Striped T-shirt', 'image_url': 'https://res.cloudinary.com/driwu0lv4/image/upload/v1759947780/product_matcher/products/12947.jpg', 'similarity': array([[1.]])}


In [104]:
for i,m in enumerate(matches[:10],1):
  if m['similarity'] > 0.5:
    print(f"{i}. {m['name']} - {m['similarity']}")

1. Basics Men Blue Striped T-shirt - [[1.]]
2. Probase Men Printed Black Tshirts - [[0.83591048]]
3. Femella Women Sea Green Printed Top - [[0.80242943]]
4. Genesis Black Casual Shirt - [[0.78322139]]
5. Facit Men Bicep Tee Red Innerwear Vest - [[0.78202236]]
6. Levis Men Comfort Style Grey Innerwear Vest - [[0.7711838]]
7. Femella Women Purple & Black Dress - [[0.76732873]]
8. Puma Men Blue & White T-shirt - [[0.75922184]]
9. French Connection Women Printed Red Dress - [[0.75008986]]
10. United Colors of Benetton Women Black Top - [[0.74621543]]
